# Pré-processamento dos dados

In [1]:
import pandas as pd
import os
import zlib
import math
import numpy as np
import random

In [2]:
# introdução manual da data a que os dados se referem
data = '2022-10'

## 1. Listagem informação dos indicadores 

In [3]:
# 'scrapped_indicadores.csv' resulta do webscraping da pagina SDM com a informação de cada indicador
# 'scrapped_intervalos.csv' resulta da extração dos intervalos aceitáveis e esperado do documento de Operacionalização dos CSP 2022, paginas 72-76

# importação das duas listas com a informação a juntar
indicadores = pd.read_csv('scrapped_indicadores.csv')
intervalos = pd.read_csv('scrapped_intervalos.csv')

# remover colunas repetidas
intervalos = intervalos.drop(columns=['nome_indicador'])

# juntar os dois data frames sem perder informação
indicadores = pd.merge(indicadores,intervalos, on = ['id'], how = 'outer')

# alterar nome de 'id'
indicadores = indicadores.rename(columns={'id':'id_indicador'})
indicadores['id_indicador'] = indicadores['id_indicador'].astype(int)


# indicadores com impacto no IDG
impacto_idg = pd.read_csv('usf_ucsp_indicadores_2022_comimpactoIDG.csv')
impacto_idg['conta_IDG'] = True
impacto_idg = impacto_idg.rename(columns={'indicador':'id_indicador'})
#impacto_idg['id_indicador'] = impacto_idg['id_in#dicador'].astype(int)

#impacto_idg
#impacto_idg.duplicated().sum()
#indicadores.duplicated().sum()

#indicadores = indicadores.merge(impacto_idg, on='id_indicador', how='outer')
indicadores_merged = indicadores.merge(impacto_idg, on='id_indicador', how='outer')

indicadores_merged['conta_IDG'].fillna(value='False', inplace=True)
#print(indicadores.columns)
#print(impacto_idg.columns)

# gravar em csv
indicadores_merged.to_csv('LISTA_INDICADORES.csv', header = 1, index=False)

#indicadores.head(10)
indicadores_merged.head(10)

,id_indicador,codigo,codigo_siars,nome_abreviado,designacao,objetivo,formula,unidade_de_medida,output,estado_do_indicador,...,inclusao_de_utentes_no_indicador,prazo_para_registos,link,min_aceitavel,min_esperado,max_esperado,max_aceitavel,intervalo_aceitavel,intervalo_esperado,conta_IDG
0,1,3.12.01,2013.001.01,Proporção de consultas realizadas pelo MF,Proporção de consultas realizadas pelo respeti...,Monitorizar o acesso dos utentes ao seu própri...,AA / BB x 100,%,Proporção de contactos,Com dados desde Dezembro de 2012,...,Utentes inscritos em pelo menos um dia do perí...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=001&CLUSTE...,75.0,78.0,90.0,92.0,[75; 92],[78; 90],True
1,2,3.15.01,2013.002.01,Taxa de utilização global de consultas médicas,Taxa de utilização global de consultas médicas,Avaliar o acesso a consultas médicas pela popu...,AA / BB x 100,%,Proporção de utentes,Com dados desde Dezembro de 2012,...,Utentes inscritos à data de referência do indi...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=002&CLUSTE...,66.0,73.5,85.0,90.0,[66; 90],[73.5; 85],False
2,3,4.18.01,2013.003.01,Taxa de domicílios médicos por 1.000 inscritos,Taxa de consultas médicas no domicílio por 1.0...,Permite monitorizar produtividade relacionada ...,AA / BB x 1.000,por 1.000,Taxa de consultas domiciliárias (Méd.),Com dados desde Dezembro de 2012,...,Utentes inscritos em pelo menos um dia do perí...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=003&CLUSTE...,12.0,18.0,35.0,40.0,[12; 40],[18; 35],True
3,4,4.30.01,2013.004.01,Taxa de domicílios enfermagem por 1.000 inscritos,Taxa de consultas de enfermagem no domicílio p...,Permite monitorizar produtividade relacionada ...,AA / BB x 1.000,por 1.000,Taxa de consultas domiciliárias (Enf.),Com dados desde Dezembro de 2012,...,Utentes inscritos em pelo menos um dia do perí...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=004&CLUSTE...,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5,3.12.02,2013.005.01,Proporção de consultas realizadas pelo EF,Proporção de consultas realizadas pelo respeti...,Monitorizar o acesso dos utentes ao seu própri...,AA / BB x 100,%,Proporção de contactos,Com dados desde Dezembro de 2012,...,Utentes inscritos em pelo menos um dia do perí...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=005&CLUSTE...,65.0,70.0,88.0,92.0,[65; 92],[70; 88],True
5,6,3.15.02,2013.006.01,Taxa de utilização de consultas médicas - 3 anos,Taxa de utilização global de consultas médicas...,Avaliar o acesso a consultas médicas pela popu...,AA / BB x 100,%,Proporção de utentes,Com dados desde Dezembro de 2012,...,Utentes inscritos à data de referência do indi...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=006&CLUSTE...,75.0,88.0,100.0,100.0,[75; 100],[88; 100],True
6,7,7.10,2013.007.01,Proporção utiliz. referenciados p/ consulta hosp.,Proporção de utilizadores referenciados para c...,Monitorizar a taxa de referenciação hospitalar...,AA / BB x 100,%,Proporção de utilizadores,Com dados desde Dezembro de 2012,...,Utentes utilizadores durante o período em análise,5 dias,https://sdm.min-saude.pt/BI.aspx?id=007&CLUSTE...,NaN,NaN,NaN,NaN,NaN,NaN,False
7,8,3.22.01,2013.008.01,Taxa de utilização de consultas de PF (méd./enf.),Taxa de utilização de consultas de planeamento...,Monitorizar a utilização das consultas de saúd...,AA / BB x 100,%,Proporção de mulheres,Com dados desde Dezembro de 2012,...,Utentes inscritos à data de referência do indi...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=008&CLUSTE...,38.0,60.0,100.0,100.0,[38; 100],[60; 100],True
8,9,3.22.02,2013.009.01,Taxa de utilização de consultas de PF (enf.),Taxa de utilização de consultas de enfermagem ...,Monitorizar a utilização das consultas de enfe...,AA / BB x 100,%,Proporção de mulheres,Com dados desde Dezembro de 2012,...,Utentes inscritos à data de referência do indi...,5 dias,https://sdm.min-saude.pt/BI.aspx?id=009&CLUSTE...,25.0,42.0,75.0,80.0,[25; 80],[42; 75],False
9,10,3.22.03,2013.010.01,Taxa de utilização de consultas de PF (méd.),Taxa de utilização de consultas médicas de

## 2. Lista valores de indicadores da unidade

In [4]:
# importação da lista com o valor de cada indicador, numerador/denominador, por médico de família
# listagem importada do MIM@UF relativo a dados de outubro de 2022
df = pd.read_excel('P02_01_R03_ Indicadores por lista de utentes de médico_corrigido.xlsx', skiprows=7, header=1)

# remover a primeira linha, sem informação relevante, e reposição da numeração index
df.drop(df.index[0], inplace=True)
df.reset_index(inplace=True, drop=True)

# remover colunas não úteis
df = df.drop(columns = ['Unidade Funcional / Polo Hospitalar', 'Indicador', 'Tipo FX/FL', 'Mês', df.columns[6]])

# mudança do nome de colunas
df = df.rename(columns={'Unnamed: 3': 'Código SIARS', 'Unnamed: 4': 'nome_abreviado', 'Médico Familia':'Médico Família', '2022-10':'Numerador','2022-10.1':'Denominador', '2022-10.2': 'Valor' })

# tirar as linhas 'Sem Médico'
df = df[df['Médico Família'] != 'Sem Médico']
df = df[df['Médico Família'] != 'MÉDICO DESCONHECIDO']

# Mudar para iniciais do MF
df['Médico Família'] = df['Médico Família'].apply(lambda x: x.split()[0][0] + x.split()[-1][0])

# extração do código do indicador do 'Código SIARS'
df['id_indicador'] = df['Código SIARS'].str.extract(r'\.(\d+)\.', expand=True)
df['id_indicador'] = df['id_indicador'].str.lstrip('0')
df['id_indicador'] = df['id_indicador'].astype(int)

# extração do FL/FX do 'Código SIARS'
df[['Código SIARS','FX/FL']] = df['Código SIARS'].str.split(' ', expand=True)

# remover mais colunas não úteis
df = df.drop(columns = ['Código SIARS', 'nome_abreviado'])

# adição do ano e mês referente aos dados
df['Data'] = data

# converter texto em float e int, e arredondar o valor com 1 casa decimal
df['Valor'] = df['Valor'].astype(float).round(1)
df['Numerador'] = df['Numerador'].astype(int)
df['Denominador'] = df['Denominador'].astype(int)

# reorganização da ordem das colunas
df = df.reindex(columns=['id_indicador', 'FX/FL', 'Data', 'Médico Família', 'Numerador', 'Denominador','Valor'])

# gravar em .csv
#df.to_csv('INDICADORES_POR_MÉDICO_10_2022.csv', header = 1, index=False)



####

#removido

'''df_unidade = df.drop(columns = ['Médico Família','Valor'])

df_unidade = df_unidade.groupby(['id_indicador', 'FX/FL', 'Data']).sum(['Numerador', 'Denominador']).reset_index()

df_unidade['Médico Família'] = 'Unidade'

df_unidade['Valor'] = 100 * df_unidade['Numerador'] / df_unidade['Denominador']
df_unidade['Valor'] = df_unidade['Valor'].round(1)

df_unidade = df_unidade.reindex(columns=['id_indicador', 'FX/FL', 'Data', 'Médico Família', 'Numerador', 'Denominador','Valor'])

# Juntar dados da unidade aos de cada médico
df = pd.concat([df_unidade,df])'''

####

# Agora feature engeneering
df_indicador_medico = df

# extrarir apenas as  colunas necessárias com os intervalos
df_intervalos_indicadores = indicadores[['id_indicador','nome_abreviado', 'unidade_de_medida', 'min_aceitavel', 'min_esperado', 'max_esperado', 'max_aceitavel']]

# juntar as duas listas
df_indicador_medico = df_indicador_medico.merge(df_intervalos_indicadores, on='id_indicador')

df_indicador_medico['Classificação'] = df_indicador_medico.apply(
    lambda row:
        '2' if row['Valor'] >= row['min_esperado'] and row['Valor'] <= row['max_esperado']
        else '1-' if row['Valor'] >= row['min_aceitavel'] and row['Valor'] <= row['min_esperado']
        else '1+' if row['Valor'] >= row['max_esperado'] and row['Valor'] <= row['max_aceitavel']
        else '0-' if row['Valor'] < row['min_aceitavel']
        else '0+' if row['Valor'] > row['max_aceitavel']
        else np.nan
    , axis=1)


def unidade_medida_divisao(unidade_de_medida):
    if unidade_de_medida == '%': return 100
    elif unidade_de_medida == 'Média': return 16
    elif unidade_de_medida == 'dias': return 1
    elif unidade_de_medida == 'por 1.000': return 1000
    elif unidade_de_medida == 'por 10.000': return 10000
    elif unidade_de_medida == 'Índice (escala valores entre 0 e 1)': return 1
    elif unidade_de_medida == 'Índice (escala valores entre 0 e aproximadamente 1)': return 2
    elif unidade_de_medida == '€ / UTE': return 1
    elif unidade_de_medida == '€ / UTI': return 1
    else: return 100

#
df_indicador_medico['Numerador_minimo_aceitavel'] = df_indicador_medico.apply(
    lambda row:
        row['Denominador'] * row['min_aceitavel'] / unidade_medida_divisao(row['unidade_de_medida'])
    , axis=1)
df_indicador_medico['Numerador_minimo_aceitavel'] = df_indicador_medico['Numerador_minimo_aceitavel'].round(1)


#
df_indicador_medico['Numerador_minimo_esperado'] = df_indicador_medico.apply(
    lambda row:
        row['Denominador'] * row['min_esperado'] / unidade_medida_divisao(row['unidade_de_medida'])
    , axis=1)
df_indicador_medico['Numerador_minimo_esperado'] = df_indicador_medico['Numerador_minimo_esperado'].round(1)


#
df_indicador_medico['Numerador_maximo_esperado'] = df_indicador_medico.apply(
    lambda row:
        row['Denominador'] * row['max_esperado'] / unidade_medida_divisao(row['unidade_de_medida'])
    , axis=1)
df_indicador_medico['Numerador_maximo_esperado'] = df_indicador_medico['Numerador_maximo_esperado'].round(1)


#
df_indicador_medico['Numerador_maximo_aceitavel'] = df_indicador_medico.apply(
    lambda row:
        row['Denominador'] * row['max_aceitavel'] / unidade_medida_divisao(row['unidade_de_medida'])
    , axis=1)
df_indicador_medico['Numerador_maximo_aceitavel'] = df_indicador_medico['Numerador_maximo_aceitavel'].round(1)

#
df_indicador_medico['Até_alvo_esperado'] = df_indicador_medico.apply(
    lambda row:
        0 if row['Valor'] >= row['min_esperado'] and row['Valor'] <= row['max_esperado']
        else row['Numerador_minimo_esperado'] - row['Numerador'] if row['Valor'] < row['min_esperado']
        else row['Numerador_minimo_esperado'] - row['Numerador'] if row['Valor'] > row['max_esperado']
        else np.nan
    , axis=1)
df_indicador_medico['Até_alvo_esperado'] = df_indicador_medico['Até_alvo_esperado'].round(1)

#
df_indicador_medico['Até_alvo_aceitavel'] = df_indicador_medico.apply(
    lambda row:
        0 if row['Valor'] >= row['min_aceitavel'] and row['Valor'] <= row['max_aceitavel']
        else row['Numerador_minimo_aceitavel'] - row['Numerador'] if row['Valor'] < row['min_aceitavel']
        else row['Numerador_minimo_aceitavel'] - row['Numerador'] if row['Valor'] > row['max_aceitavel']
        else np.nan
    , axis=1)
df_indicador_medico['Até_alvo_aceitavel'] = df_indicador_medico['Até_alvo_aceitavel'].round(1)

# codigo para formatação condicional cores
df_indicador_medico['codigo_cor'] = df_indicador_medico.apply(
    lambda row:
        1 if row['Classificação'] == '0-'
        else 2 if row['Classificação'] == '1-'
        else 3 if row['Classificação'] == '2'
        else 4 if row['Classificação'] == '1+'
        else 5 if row['Classificação'] == '0+'
        else 0
    , axis=1)
df_indicador_medico['codigo_cor'] = df_indicador_medico['codigo_cor'].astype(int)


# Adicionar alvo mais próximo e a etiqueta correspondente para o Gauge
df_indicador_medico['alvo'] = df_indicador_medico.apply(
    lambda row:
        row['min_aceitavel'] if row['Classificação'] == '0-'
        else row['min_esperado'] if row['Classificação'] == '1-'
        else row['min_esperado'] if row['Classificação'] == '2'
        else row['max_esperado'] if row['Classificação'] == '1+'
        else row['max_aceitavel'] if row['Classificação'] == '0+'
        else 0
    , axis=1)

df_indicador_medico['alvo_etiqueta'] = df_indicador_medico.apply(
    lambda row:
        'Mínimo Aceitável' if row['Classificação'] == '0-'
        else 'Mínimo Esperado' if row['Classificação'] == '1-'
        else 'No Alvo' if row['Classificação'] == '2'
        else 'Máximo Esperado' if row['Classificação'] == '1+'
        else 'Máximo Aceitável' if row['Classificação'] == '0+'
        else 'N/A'
    , axis=1)

    
#remover colunas desnecessárias após calculos
df_indicador_medico = df_indicador_medico.drop(columns = ['nome_abreviado'])

# gravar em csv
df_indicador_medico.to_csv('INDICADORES_POR_MÉDICO.csv', header = 1, index=False)
#df_indicador_medico.head(50)

#set(df_indicador_medico['unidade_de_medida'])

# para ver que indicadores ficaram na lista
#df_indicador_medico_dropna = df_indicador_medico.dropna()
#set(df_indicador_medico_dropna['id_indicador'].tolist())

df_indicador_medico.head(10)

,id_indicador,FX/FL,Data,Médico Família,Numerador,Denominador,Valor,unidade_de_medida,min_aceitavel,min_esperado,...,Classificação,Numerador_minimo_aceitavel,Numerador_minimo_esperado,Numerador_maximo_esperado,Numerador_maximo_aceitavel,Até_alvo_esperado,Até_alvo_aceitavel,codigo_cor,alvo,alvo_etiqueta
0,2,FL,2022-10,FA,1205,1805,66.8,%,66.0,73.5,...,1-,1191.3,1326.7,1534.2,1624.5,121.7,0.0,2,73.5,Mínimo Esperado
1,2,FL,2022-10,RB,1159,1747,66.3,%,66.0,73.5,...,1-,1153.0,1284.0,1485.0,1572.3,125.0,0.0,2,73.5,Mínimo Esperado
2,2,FL,2022-10,PM,1354,1864,72.6,%,66.0,73.5,...,1-,1230.2,1370.0,1584.4,1677.6,16.0,0.0,2,73.5,Mínimo Esperado
3,2,FL,2022-10,JP,1246,1717,72.6,%,66.0,73.5,...,1-,1133.2,1262.0,1459.4,1545.3,16.0,0.0,2,73.5,Mínimo Esperado
4,2,FL,2022-10,SC,1227,1841,66.6,%,66.0,73.5,...,1-,1215.1,1353.1,1564.8,1656.9,126.1,0.0,2,73.5,Mínimo Esperado
5,2,FL,2022-10,SB,1181,1819,64.9,%,66.0,73.5,...,0-,1200.5,1337.0,1546.2,1637.1,156.0,19.5,1,66.0,Mínimo Aceitável
6,2,FL,2022-10,SL,1226,1878,65.3,%,66.0,73.5,...,0-,1239.5,1380.3,1596.3,1690.2,154.3,13.5,1,66.0,Mínimo Aceitável
7,2,FL,2022-10,SG,1248,1771,70.5,%,66.0,73.5,...,1-,1168.9,1301.7,1505.4,1593.9,53.7,0.0,2,73.5,Mínimo Esperado
8,2,FL,2022-10,JS,1178,1759,67.0,%,66.0,73.5,...,1-,1160.9,1292.9,1495.2,1583.1,114.9,0.0,2,73.5,Mínimo Esperado
9,2,FL,2022-10,CG,1318,1825,72.2,%,66.0,73.5,...,1-,1204.5,1341.4,1551.2,1642.5,23.4,0.0,2,73.5,Mínimo Esperado


## 3. Lista de cumpridores de indicadores DM

In [5]:
# Set the directory path
directorio_indicadores = 'indicadores'

# Get the list of items in the directory
items_indicadores = os.listdir(directorio_indicadores)

# Filter the list to only include files
files_indicadores = [item for item in items_indicadores if os.path.isfile(os.path.join(directorio_indicadores, item))]
if files_indicadores[0] == '.DS_Store':
    files_indicadores.pop(0)


# pandas DataFrame para gravar durante iteração
df_indicadores = pd.DataFrame()

for file in files_indicadores:
    #df = indicarores_pre_processing(directory +'/', file)
    #df = pd.read_excel(directory + '/' + file, skiprows=8, header=0)
    
    # limpar as 8 linhas iniciais
    df = pd.read_excel(directorio_indicadores + '/' + file, skiprows=8, header=1)

    filename, ext = os.path.splitext(file)
    
    # mudar nomes de colunas
    df = df.rename(columns={df.columns[8]: 'Enfermeiro Família', 'Médico Familia':'Médico Família' })

    # Transfomrar nome em iniciais para efeitos de privacidade
    df["Iniciais"] = ''
    
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Split the name into separate parts
        parts = row["Utente"].split()

        # Convert the parts into initials
        initials = [part[0] for part in parts]

        # Join the initials into a single string
        initials = "".join(initials)

        # Update the Initials column with the initials
        df.at[index, "Iniciais"] = initials

    # remover o upper case para inicias em caps e transformar em siglas de 2 letras
    df['Enfermeiro Família'] = df['Enfermeiro Família'].str.title()
    df['Médico Família'] = df['Médico Família'].str.title()
    df['MF'] = df['Médico Família'].apply(lambda x: x.split()[0][0] + x.split()[-1][0])
    df['EF'] = df['Enfermeiro Família'].apply(lambda x: x.split()[0][0] + x.split()[-1][0])

    # adicionar nova coluna com a info sobre indicador
    df['id_indicador'] = filename
    df['FX/FL'] = filename[-2:]
    df['id_indicador'] = df['id_indicador'].str.extract(r'(\d+)')
    df['id_indicador'] = df['id_indicador'].astype(int)
    
    df['Sexo'] = df['Sexo'].apply(lambda x: x[0])
    
    # remover 'Anos' e 'Ano' de idade
    df['Idade'] = df['Idade'].str.extract(r'(\d+)')
    
    # adição do ano e mês referente aos dados
    df['Data'] = data 

    # remover colunas 
    df = df.drop(columns=['Mês', 'Freguesia Habitação', 'Mobilidade', 'Enfermeiro (Nome+Cédula)', 'Utente', 'Médico Família', 'Enfermeiro Família'])
    df = df.rename(columns={'EF': 'Enfermeiro Família', 'MF':'Médico Família' })
    
    # id_utente a partir do NOP, Iniciais do nome e Sexo, por um algoritmo de hashing (o MF e a idade pode mudar nos registos extraidos do MIM@UF)
    df['id_utente'] = df.apply(lambda row: zlib.adler32(bytes(str(row['NOP']) + str(row['Iniciais']) + str(row['Sexo']), 'utf-8')), axis=1)
    
    #Trocar CUMPRE/ÃO CUMPRE por Sim/Não excepto no 314 que está trocado...
    df = df.rename(columns={'Cumprimento':'Cumpre?'})
    if filename == '314_FL':
        df['Cumpre?'] = df['Cumpre?'].apply(
            lambda row:
                'Não' if row == 'CUMPRE'
                else 'Sim'
        )
        
        #Código de cores
        df['Cumpre_cores'] = df['Cumpre?'].apply(
        lambda row:
            1 if row == 'Não'
            else 0
        )
    else:
        df['Cumpre?'] = df['Cumpre?'].apply(
            lambda row:
                'Sim' if row == 'CUMPRE'
                else 'Não'
        )
        
        #Código de cores
        df['Cumpre_cores'] = df['Cumpre?'].apply(
        lambda row:
            1 if row == 'Sim'
            else 0
        )
    
    # reorganizar ordem de colunas
    df = df.reindex(columns=['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família', 'Enfermeiro Família', 'id_indicador','FX/FL', 'Data', 'Cumpre?', 'Cumpre_cores'])
    
    # juntar a uma dataframe principal
    
    df_indicadores = pd.concat([df_indicadores, df])
    
    # opção de gravar cada um em csv
    #df.to_csv(filename + '.csv', header = 1, index=False)

# criação de uma lista de utentes para separar informação
lista_utentes = df_indicadores[['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família', 'Enfermeiro Família']]
lista_utentes = lista_utentes.drop_duplicates()

# gravar em csv
#lista_utentes.to_csv('LISTA_UTENTES.csv', header = 1, index=False)

# remover colunas não uteis
df_indicadores = df_indicadores.drop(columns=['NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família', 'Enfermeiro Família'])


# gravar em csv
df_indicadores.to_csv('CUMPRIDORES_INDICADORES.csv', header = 1, index=False)

df_indicadores.head(10)

,id_utente,id_indicador,FX/FL,Data,Cumpre?,Cumpre_cores
0,159842833,36,FX,2022-10,Sim,1
1,204341875,36,FX,2022-10,Não,0
2,167379499,36,FX,2022-10,Sim,1
3,209060461,36,FX,2022-10,Sim,1
4,357434211,36,FX,2022-10,Sim,1
5,199230041,36,FX,2022-10,Sim,1
6,245236385,36,FX,2022-10,Sim,1
7,201196117,36,FX,2022-10,Sim,1
8,248971960,36,FX,2022-10,Não,0
9,199426649,36,FX,2022-10,Sim,1


## 4. Lista de MCDT's

In [34]:
# Set the directory path
directorio_mcdts = 'mcdts'

# Get the list of items in the directory
items_mcdts = os.listdir(directorio_mcdts)

# Filter the list to only include files
files_mcdts = [item for item in items_mcdts if os.path.isfile(os.path.join(directorio_mcdts, item))]
if files_mcdts[0] == '.DS_Store':
    files_mcdts.pop(0)

df_mcdts = pd.DataFrame()

for file in files_mcdts:
    # Limpar as 7 linhas iniciais
    df = pd.read_excel(directorio_mcdts + '/' + file, skiprows=7, header=1)

    filename, ext = os.path.splitext(file)
    
    # Mudar nomes de colunas
    df = df.rename(columns={df.columns[2]: 'Nome', 'Médico Familia':'Médico Família'})

    # Transfomrar nome em iniciais para efeitos de privacidade
    df["Iniciais"] = ''
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Split the name into separate parts
        parts = row["Nome"].split()

        # Convert the parts into initials
        initials = [part[0] for part in parts]

        # Join the initials into a single string
        initials = "".join(initials)

        # Update the Initials column with the initials
        df.at[index, "Iniciais"] = initials

    # Transformar sexo em iniciais
    df['Sexo'] = df['Sexo'].apply(lambda x: x[0])
        
    # Remover 'Anos' e 'Ano' de idade
    df['Idade'] = df['Idade'].str.extract(r'(\d+)')

    # Remover o upper case para inicias em caps
    df['Médico Família'] = df['Médico Família'].str.title()
    df['Médico Família'] = df['Médico Família'].apply(lambda x: x.split()[0][0] + x.split()[-1][0])

    df['MCDT'] = filename.split('_')[0]
    
    # Remover colunas 
    df = df.drop(columns=['Freguesia Habitação', 'Utente', 'Nome', 'Utente'])
    
    df['Resultado Último MCDT'] = df['Resultado Último MCDT'].round(2)

    # id_utente a partir do NOP, Iniciais do nome e Sexo, por um algoritmo de hashing (o MF e a idade pode mudar nos registos extraidos do MIM@UF)
    df['id_utente'] = df.apply(lambda row: zlib.adler32(bytes(str(row['NOP']) + str(row['Iniciais']) + str(row['Sexo']), 'utf-8')), axis=1)
    
    # Reorganizar ordem de colunas
    df = df.reindex(columns=['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família', 'MCDT', 'Data Último MCDT','Resultado Último MCDT'])
    
    # opção de gravar cada um em csv
    df.drop(columns=['NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família']).to_csv(filename + '.csv', header = 1, index=False)
    
    df_mcdts = pd.concat([df_mcdts, df])

# criação de uma lista de utentes para separar informação
lista_utentes_2 = df_mcdts[['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família']]
lista_utentes_2 = lista_utentes_2.drop_duplicates()

# remover colunas não uteis
df_mcdts = df_mcdts.drop(columns=['NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família'])

# calcular LDL com base no id_utente e na data do MCDT
df_ctotal = df_mcdts[df_mcdts['MCDT']=='ColesterolTotal']
df_hdl = df_mcdts[df_mcdts['MCDT']=='HDL']
df_trig = df_mcdts[df_mcdts['MCDT']=='Trigliceridos']

# Adicionar Ctotal
df_ldl = df_ctotal.rename(columns={'Resultado Último MCDT': 'Colesterol total'})

# Adicionar HDL
df_ldl = df_ldl.merge(df_hdl, on=['id_utente','Data Último MCDT'])
df_ldl = df_ldl.rename(columns={'Resultado Último MCDT': 'HDL'})

# Adicionar Trig
df_ldl = df_ldl.merge(df_trig, on=['id_utente','Data Último MCDT'])
df_ldl = df_ldl.rename(columns={'Resultado Último MCDT': 'Trig'})

# remover colunas que não interessam
df_ldl = df_ldl.drop(columns=['MCDT_x', 'MCDT_y'])

# Fórmula calculo LDL
df_ldl['Resultado Último MCDT'] = df_ldl['Colesterol total'] - df_ldl['HDL'] - (df_ldl['Trig'] / 5)

# trocar a etiqueta
df_ldl['MCDT'] = 'LDL'

# Arredondar a 1 casas decimais
df_ldl['Resultado Último MCDT'] = df_ldl['Resultado Último MCDT'].round(1)

# Alterara a ordem das colunas
df_ldl = df_ldl.reindex(columns= ['id_utente', 'MCDT', 'Data Último MCDT', 'Resultado Último MCDT'])

# juntar aos outros MCDTS
df_mcdts = pd.concat([df_mcdts, df_ldl])

# gravar
#df_mcdts.to_csv('MCDTS.csv', header = 1, index=False)

df_ldl.to_csv('LDL_2022_10' + '.csv', header = 1, index=False)
df_ldl

,id_utente,MCDT,Data Último MCDT,Resultado Último MCDT
0,174129735,LDL,2022-02-11,57.8
1,168231461,LDL,2021-01-23,68.8
2,217711245,LDL,2022-02-09,37.8
3,171377200,LDL,2022-07-27,74.0
4,216859269,LDL,2022-04-22,153.4
...,...,...,...,...
9693,348455735,LDL,2022-08-12,99.6
9694,296616698,LDL,2022-03-11,113.2
9695,338821925,LDL,2022-01-10,111.8
9696,242942623,LDL,2020-11-05,124.2


## 5. Lista de Biometrias

In [35]:
# Set the directory path
directorio_biomarcadores = 'biomarcadores'

# Get the list of items in the directory
items_biomarcadores = os.listdir(directorio_biomarcadores)

# Filter the list to only include files
files_biomarcadores = [item for item in items_biomarcadores if os.path.isfile(os.path.join(directorio_biomarcadores, item))]
if files_biomarcadores[0] == '.DS_Store':
    files_biomarcadores.pop(0)

    
df_biomarcadores = pd.DataFrame()

for file in files_biomarcadores:
    # Limpar as 7 linhas iniciais
    df = pd.read_excel(directorio_biomarcadores + '/' + file, skiprows=7, header=1)

    filename, ext = os.path.splitext(file)
    
    # Mudar nomes de colunas
    df = df.rename(columns={df.columns[2]: 'Nome', 'Médico Familia':'Médico Família'})

    # Transfomrar nome em iniciais para efeitos de privacidade
    df["Iniciais"] = ''
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Split the name into separate parts
        parts = row["Nome"].split()

        # Convert the parts into initials
        initials = [part[0] for part in parts]

        # Join the initials into a single string
        initials = "".join(initials)

        # Update the Initials column with the initials
        df.at[index, "Iniciais"] = initials

    # Transformar sexo em iniciais
    df['Sexo'] = df['Sexo'].apply(lambda x: x[0])
        
    # Remover 'Anos' e 'Ano' de idade
    df['Idade'] = df['Idade'].str.extract(r'(\d+)')

    # Remover o upper case para inicias em caps
    df['Médico Família'] = df['Médico Família'].str.title()
    df['Médico Família'] = df['Médico Família'].apply(lambda x: x.split()[0][0] + x.split()[-1][0])

    df['Biomarcador'] = filename.split('_')[0]
    
    # Remover colunas 
    df = df.drop(columns=['Freguesia Habitação', 'Utente', 'Nome', 'Utente'])

    # id_utente a partir do NOP, Iniciais do nome e Sexo, por um algoritmo de hashing (o MF e a idade pode mudar nos registos extraidos do MIM@UF)
    df['id_utente'] = df.apply(lambda row: zlib.adler32(bytes(str(row['NOP']) + str(row['Iniciais']) + str(row['Sexo']), 'utf-8')), axis=1)
    
    # Reorganizar ordem de colunas
    df = df.reindex(columns=['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família', 'Biomarcador', 'Data Último Registo','Resultado Último Registo'])

    # opção de gravar cada um em csv
    df.drop(columns=['NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família']).to_csv(filename + '.csv', header = 1, index=False)
    
    df_biomarcadores = pd.concat([df_biomarcadores, df])

# criação de uma lista de utentes para separar informação
lista_utentes_3 = df_biomarcadores[['id_utente', 'NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família']]
lista_utentes_3 = lista_utentes_3.drop_duplicates()

# remover colunas não uteis
df_biomarcadores = df_biomarcadores.drop(columns=['NOP', 'Iniciais', 'Idade', 'Sexo', 'Médico Família'])

# gravar
#df_biomarcadores.to_csv('BIOMARCADORES.csv', header = 1, index=False)
df_biomarcadores

,id_utente,Biomarcador,Data Último Registo,Resultado Último Registo
0,174129735,TAs,2022-04-11,126.0
1,168231461,TAs,2022-09-01,120.0
2,217711245,TAs,2022-10-21,110.0
3,171377200,TAs,2022-02-04,140.0
4,216859269,TAs,2022-06-30,130.0
...,...,...,...,...
13191,338821925,TAd,2022-10-10,82.0
13192,242942623,TAd,2022-06-17,69.0
13193,289669865,TAd,2022-06-01,62.0
13194,432735107,TAd,2022-10-04,87.0


## 6. Registos globais

In [36]:
## Registos (unificação de biomarcadores e MCDT's)

### NÃO UTILIZADO ###
'''df_biomarcadores = df_biomarcadores.rename(columns={
    'Biomarcador':'Tipo registo',
    'Data Último Registo': 'Data',
    'Resultado Último Registo': 'Resultado'})

df_mcdts = df_mcdts.rename(columns={
    'MCDT':'Tipo registo',
    'Data Último MCDT': 'Data',
    'Resultado Último MCDT': 'Resultado'})

df_registos = pd.concat([df_biomarcadores, df_mcdts])

df_registos.to_csv('registos.csv', header = 1, index=False)'''


"df_biomarcadores = df_biomarcadores.rename(columns={\n    'Biomarcador':'Tipo registo',\n    'Data Último Registo': 'Data',\n    'Resultado Último Registo': 'Resultado'})\n\ndf_mcdts = df_mcdts.rename(columns={\n    'MCDT':'Tipo registo',\n    'Data Último MCDT': 'Data',\n    'Resultado Último MCDT': 'Resultado'})\n\ndf_registos = pd.concat([df_biomarcadores, df_mcdts])\n\ndf_registos.to_csv('registos.csv', header = 1, index=False)"

## 7. Lista de Utentes

In [37]:
# Juntar a listas de utentes das várias fontes numa unica lista
#lista_utentes = pd.concat([lista_utentes, lista_utentes_2, lista_utentes_3])

# remover duplicados (há linhas com a mesma pessoa mas com idade diferente)
lista_utentes = lista_utentes.drop_duplicates(subset=['id_utente'])

# Iniciais nome aleatórias 
lista_utentes['Iniciais'] = lista_utentes['Iniciais'].apply(lambda x: ''.join(random.choices('ABCDEFGHIJLMNOPQRSTUVXZ', k=4)))

# 
lista_utentes = lista_utentes.drop(columns='NOP')

# Converter idade para int
lista_utentes['Idade'] = lista_utentes['Idade'].astype(int)

# Intervalos para histograma
lista_utentes['Idade_q'] = lista_utentes['Idade'].apply(
    lambda row: 
        '<5' if row <5
        else '5-9' if row <10
        else '10-14' if row <15
        else '15-19' if row <20
        else '20-24' if row <25
        else '25-29' if row <30
        else '30-34' if row <35
        else '35-39' if row <40
        else '40-44' if row <45
        else '45-49' if row <50
        else '50-54' if row <55
        else '55-59' if row <60
        else '60-64' if row <65
        else '65-69' if row <70
        else '70-74' if row <75
        else '75-79' if row <80
        else '80-84' if row <85
        else '85-89' if row <90
        else '90-94' if row <95
        else '>95'
)


# gravar em csv
lista_utentes.to_csv('LISTA_UTENTES_DM.csv', header = 1, index=False)

#lista_utentes['id_utente'].duplicated().sum()
#lista_utentes['id_utente'].duplicated()
lista_utentes

,id_utente,Iniciais,Idade,Sexo,Médico Família,Enfermeiro Família,Idade_q
0,159842833,AHIQ,67,M,SB,AS,65-69
1,204341875,BTIM,86,M,CG,TL,85-89
2,167379499,HDGZ,61,H,RB,IA,60-64
3,209060461,VQQT,66,H,FA,ML,65-69
4,357434211,MDAV,88,M,FA,ML,85-89
...,...,...,...,...,...,...,...
1565,399836034,PIEQ,72,M,SG,MD,70-74
1566,234357367,IRGV,82,H,CG,TL,80-84
1567,293208805,PGNA,75,H,AP,IA,75-79
1568,282854095,MCVV,78,M,SG,MD,75-79


## 8. Adicionar coluna se de 2022 na HbA1c, LDL, TAs e TAD + Intervalos para histogrma

In [38]:
# Importar HbA1c
df_hba1c = pd.read_csv('HbA1c_10_2022.csv')

# Filto para apenas resultados resultados 2022 (tendo em conta a forma como o indicador é medido)
df_hba1c['Resultado_2022'] = df_hba1c.apply(
    lambda row: row['Resultado Último MCDT'] if '2022' in row['Data Último MCDT'] else 0, axis=1)
#df_hba1c['Resultado_2022'] = df_hba1c['Resultado_2022'].astype(int)


df_hba1c['2022?'] = df_hba1c.apply(
    lambda row: True if '2022' in row['Data Último MCDT'] else False, axis=1)

# adicinar intervalos com base em lógica
df_hba1c['Intervalos'] = df_hba1c['Resultado_2022'].apply(
    lambda row:
        'N/A' if row == 0
        else '<5' if row <5
        else '5-5,9' if row <6
        else '6-6,9' if row <7
        else '7-7,9' if row <8
        else '8-8,9' if row <9
        else '9-9,9' if row <10
        else '>10' if row >=10
        else 'N/A'
)

df_hba1c['ordem_intervalos'] = df_hba1c['Resultado_2022'].apply(
    lambda row:
        8 if row == 0
        else 1 if row <5
        else 2 if row <6
        else 3 if row <7
        else 4 if row <8
        else 5 if row <9
        else 6 if row <10
        else 7 if row >= 10
        else 8
)

df_hba1c = df_hba1c.fillna(0)

# gravar em csv
df_hba1c.to_csv('HbA1c.csv', header = 1, index=False)





#### LDL

# Importar LDL
df_ldl = pd.read_csv('LDL_2022_10.csv')

# Filto para apenas resultados resultados 2022 (tendo em conta a forma como o indicador é medido)
df_ldl['Resultado_2022'] = df_ldl.apply(
    lambda row: row['Resultado Último MCDT'] if '2022' in row['Data Último MCDT']  else 0, axis=1)
#df_ldl['Resultado_2022'] = df_ldl['Resultado_2022'].astype(int)

df_ldl['2022?'] = df_ldl.apply(
    lambda row: True if '2022' in row['Data Último MCDT'] else False, axis=1)

# adicinar intervalos com base em lógica
df_ldl['Intervalos'] = df_ldl['Resultado_2022'].apply(
    lambda row:
        'N/A' if row == 0
        else '<55' if row <55
        else '55-69' if row <70
        else '70-99' if row <100
        else '>100' if row >=100
        else 'N/A'
)

df_ldl['ordem_intervalos'] = df_ldl['Resultado_2022'].apply(
    lambda row:
        5 if row == 0
        else 1 if row <55
        else 2 if row <70
        else 3 if row <100
        else 4 if row >=100
        else 5
)

df_ldl = df_ldl.fillna(0)

# gravar em csv
df_ldl.to_csv('LDL.csv', header = 1, index=False)




#### TAs

# importar
df_tas = pd.read_csv('TAs_10_2022.csv')

# Filto para apenas resultados resultados 2022 (tendo em conta a forma como o indicador é medido)
df_tas['Resultado_2022'] = df_tas.apply(
    lambda row: row['Resultado Último Registo'] if '2022' in row['Data Último Registo'] else 0, axis=1)
#df_tas['Resultado_2022'] = df_tas['Resultado_2022'].astype(int)

df_tas['2022?'] = df_tas.apply(
    lambda row: True if '2022' in row['Data Último Registo'] else False, axis=1)

# adicinar intervalos com base em lógica
df_tas['Intervalos'] = df_tas['Resultado_2022'].apply(
    lambda row:
        'N/A' if row == 0
        else '<100' if row <100
        else '100-119' if row <120
        else '120-139' if row <140
        else '140-159' if row <160
        else '>160' if row >=160
        else 'N/A'
)

df_tas['ordem_intervalos'] = df_tas['Resultado_2022'].apply(
    lambda row:
        6 if row == 0
        else 1 if row <100
        else 2 if row <120
        else 3 if row <140
        else 4 if row <160
        else 5 if row >=160
        else 6
)

df_tas = df_tas.fillna(0)

# gravar em csv
df_tas.to_csv('TAs.csv', header = 1, index=False)




#### TAd

# importar
df_tad = pd.read_csv('TAd_10_2022.csv')

# Filto para apenas resultados resultados 2022 (tendo em conta a forma como o indicador é medido)
df_tad['Resultado_2022'] = df_tad.apply(
    lambda row: row['Resultado Último Registo'] if '2022' in row['Data Último Registo'] else 0, axis=1)
#df_tad['Resultado_2022'] = df_tad['Resultado_2022'].astype(int)

df_tad['2022?'] = df_tad.apply(
    lambda row: True if '2022' in row['Data Último Registo'] else False, axis=1)


# adicinar intervalos com base em lógica
df_tad['Intervalos'] = df_tad['Resultado_2022'].apply(
    lambda row:
        'N/A' if row == 0
        else '<60' if row <60
        else '60-69' if row <70
        else '70-79' if row <80
        else '80-89' if row <90
        else '90-99' if row <100
        else '>100' if row >=100
        else 'N/A'
)

df_tad['ordem_intervalos'] = df_tad['Resultado_2022'].apply(
    lambda row:
        7 if row == 0
        else 1 if row <60
        else 2 if row <70
        else 3 if row <80
        else 4 if row <90
        else 5 if row <100
        else 6 if row >=100
        else 7
)

df_tad = df_tad.fillna(0)

# gravar em csv
df_tad.to_csv('TAd.csv', header = 1, index=False)

df_hba1c.head(50)

,id_utente,MCDT,Data Último MCDT,Resultado Último MCDT,Resultado_2022,2022?,Intervalos,ordem_intervalos
0,174129735,HbA1c,2021-01-23,91.0,0.0,False,N/A,8
1,181797464,HbA1c,2022-01-22,5.7,5.7,True,"5-5,9",2
2,159842833,HbA1c,2022-08-23,7.6,7.6,True,"7-7,9",4
3,170000955,HbA1c,2022-03-04,6.0,6.0,True,"6-6,9",3
4,204341875,HbA1c,2022-07-04,6.0,6.0,True,"6-6,9",3
5,260702936,HbA1c,2021-09-28,5.9,0.0,False,N/A,8
6,167379499,HbA1c,2022-09-13,6.1,6.1,True,"6-6,9",3
7,209060461,HbA1c,2022-07-05,6.4,6.4,True,"6-6,9",3
8,197132879,HbA1c,2020-09-03,4.9,0.0,False,N/A,8
9,357434211,HbA1c,2021-04-28,7.1,0.0,False,N/A,8
